In [33]:
import json
import pandas as pd
import time
import numpy as np
import itertools
import matplotlib.pyplot as plt 
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
import re 
from nltk.tokenize import RegexpTokenizer,word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer

In [34]:
tweets_data = []
x = []
y = []
vectorizer = CountVectorizer(stop_words = 'english',ngram_range=(1,2))
sw = set(stopwords.words('english'))
ps = PorterStemmer()

In [35]:
tweets_data_path = 'data/tweetdata.txt'
tweets_file = open(tweets_data_path, "r")
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue

In [36]:
sent = pd.read_excel('processed_data/output.xlsx')
for i in range(len(tweets_data)):
    if tweets_data[i]['id']==sent['id'][i]:
        x.append(tweets_data[i]['text'])
        y.append(sent['sentiment'][i])

In [52]:
def clean(rev):    
    p = re.findall(r"@\w+",rev)
    try:
        s = rev.replace(p[0],'')
        s = s.replace('rt :','')
    except:
        s = rev
    for i in range(len(s)):
        if s[i:i+5] == 'https':
            s = s[:i]

    t = word_tokenize(s)
    t_ = [token for token in t if token not in sw]

    word = " ".join(t_)
    return word

In [38]:
for i in range(len(x)):
    x[i] = clean(x[i])

In [39]:
for i in range(10, 15):
    print(x[i], y[i])

“ ’ think ’ going miss eighth grade . ’ tough year . lot friends struggling depr… 0
according studies , high-anxiety people likely make bad decisions tend catastrophize uncertain… -1
anyone looking organic products help ; boosting energy , depression , pain , inflammation , diabetes , sleep , . dm 🤙 0
depression 0
ur precious adorable hbahddhaddbaj 0


In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=5)

In [41]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
model = Pipeline([('vectorizer', CountVectorizer()),
 ('tfidf', TfidfTransformer()),
 ('clf', OneVsRestClassifier(LinearSVC(class_weight="balanced")))])
model2 = Pipeline([('vectorizer', CountVectorizer()),
 ('tfidf', TfidfTransformer()),
 ('clf',tree.DecisionTreeClassifier())])

In [42]:
#paramater selection
from sklearn.model_selection import GridSearchCV
parameters = {'vectorizer__ngram_range': [(1, 1), (1, 2),(2,2)],
               'tfidf__use_idf': (True, False)}
gs_clf_svm = GridSearchCV(model, parameters, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(x, y)
print(gs_clf_svm.best_score_)
print(gs_clf_svm.best_params_)

d:\mltensor\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


0.6931732933233309
{'tfidf__use_idf': False, 'vectorizer__ngram_range': (1, 1)}


In [43]:
model = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(use_idf=False)),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight="balanced")))])

In [44]:
model.fit(X_train, y_train)
#evaluation on test data
pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(pred, y_test)

array([[604, 157,  43],
       [ 81, 467,  87],
       [ 26,  72, 330]], dtype=int64)

In [59]:
model.score(X_test, y_test)

0.7504017139796465

In [46]:
def preProcess(text):
    text = [clean(text)]

In [48]:
l = "n"

In [49]:
l

'n'

In [50]:
model.predict([l])

array([-1], dtype=int64)

In [33]:
import dill as pickle
filename = 'model_v1.pk'

In [34]:
with open(filename, 'wb') as file:
	pickle.dump(model, file)